### Creating dataframe with nearest centroid data


Installing needed packages and moving to folder with files

In [ ]:
!pip install geopandas
%cd /content/drive/MyDrive/11279group34/

/content/drive/MyDrive/11279group34


In [ ]:
!pwd

/content/drive/My Drive/11279group34
centroidPoints.geojson	landuseLDI_centroids_nearestID_dist.geojson
data			README.md
demo			tests
FloridaFlow		water_centroidPoints.geojson


In [ ]:
import geopandas
import pandas as pd
import numpy as np

from scipy.spatial import cKDTree
from shapely.geometry import Point

Reading in landuse intensity geojson file <- has polygon shapes and such

In [ ]:
gdf = geopandas.read_file("data/landscapeDevelopmentIndex/Current_Landuse_-_Landscape_Development_Intensity_(LDI).geojson")

In [ ]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
gdf

,OBJECTID,LEVEL3_VALUE,LDI,LSI,WMD_DISTRICT,LANDUSE_YEAR,DESCRIPTION,SHAPEAREA,SHAPELEN,geometry,centroids
0,1,1100,6.8,3.6,SJR,2013-2016,"1100: Low Density, <2 dwelling units/acre",202486.422768,2768.478185,"POLYGON ((-80.39430 27.58675, -80.39430 27.586...",POINT (-8949496.651 3176698.682)
1,2,1100,6.8,3.6,SJR,2013-2016,"1100: Low Density, <2 dwelling units/acre",4144.953037,287.746301,"POLYGON ((-80.39067 27.58704, -80.39066 27.587...",POINT (-8949106.299 3177249.232)
2,3,1100,6.8,3.6,SJR,2013-2016,"1100: Low Density, <2 dwelling units/acre",9408.253317,471.548879,"POLYGON ((-80.42704 27.58723, -80.42662 27.587...",POINT (-8953091.749 3177318.692)
3,4,1100,6.8,3.6,SJR,2013-2016,"1100: Low Density, <2 dwelling units/acre",14729.055061,639.330146,"POLYGON ((-80.37991 27.58839, -80.37987 27.588...",POINT (-8947966.907 3177347.275)
4,5,1100,6.8,3.6,SJR,2013-2016,"1100: Low Density, <2 dwelling units/acre",81255.766776,1202.345663,"POLYGON ((-80.39454 27.58873, -80.39374 27.588...",POINT (-8949616.034 3177551.032)
...,...,...,...,...,...,...,...,...,...,...,...
1338031,1338032,6460,1.0,10.0,SJR,2013-2016,6460: Mixed Scrub-shrub Wetland,8436.737078,471.816539,"POLYGON ((-82.02071 30.72787, -82.02065 30.727...",POINT (-9130491.112 3575723.293)
1338032,1338033,6460,1.0,10.0,SJR,2013-2016,6460: Mixed Scrub-shrub Wetland,4395.450443,246.783645,"POLYGON ((-81.70438 30.72415, -81.70432 30.724...",POINT (-9095303.402 3575202.887)
1338033,1338034,6460,1.0,10.0,SJR,2013-2016,6460: Mixed Scrub-shrub Wetland,303663.108819,6475.501932,"POLYGON ((-81.81171 30.72584, -81.81178 30.726...",POINT (-9106407.185 3574920.918)
1338034,1338035,6460,1.0,10.0,SJR,2013-2016,6460: Mixed Scrub-shrub Wetland,3600.924167,238.813225,"POLYGON ((-81.73858 30.72505, -81.73852 30.725...",POINT (-9099131.919 3575314.852)


In [ ]:
gdf.to_crs(epsg=3395, inplace=True) # Find proper coordonate refrence system for datasets
gdf['centroids'] = gdf.centroid
gdf.to_crs(epsg=4326, inplace=True) # Return to previous coordonate system

In [ ]:
waterIds = [5120,5100,5110,5200,5300,5250,5400,5430,5600,6100,5710,5420,5410,
                                            6120,6130,6150,6149,6170,6159,6179,6210,6180,6189,6182,6220,6216,
                                            6250,6259,6410,6400,6420,6430,6411,6440,6460,6469,6510,6520]
water = gdf.loc[gdf["LEVEL3_VALUE"].isin(waterIds)]

Create file with just centroid of non-water objects

In [ ]:
centroidsOnly = geopandas.GeoDataFrame(gdf['OBJECTID'], geometry=gdf.centroids)
#centroidsOnly.to_file("centroidPoints.geojson", driver="GeoJSON")

centroidsOnly_water = geopandas.GeoDataFrame(water['OBJECTID'], geometry=water.centroids)
#centroidsOnly.to_file("water_centroidPoints.geojson", driver="GeoJSON")
centroidsOnly_water

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  after removing the cwd from sys.path.


,OBJECTID,geometry
593280,593281,POINT (-8915677.169 3096502.797)
593281,593282,POINT (-8927142.194 3091286.523)
593282,593283,POINT (-8917070.687 3097065.751)
593283,593284,POINT (-8916881.305 3097069.277)
593284,593285,POINT (-8916623.312 3097046.965)
...,...,...
1338031,1338032,POINT (-9130491.112 3575723.293)
1338032,1338033,POINT (-9095303.402 3575202.887)
1338033,1338034,POINT (-9106407.185 3574920.918)
1338034,1338035,POINT (-9099131.919 3575314.852)


Find nearest *water* centroid

In [ ]:
# Removing water entries from centroid list
waterParcelIDS = set(water['OBJECTID'])
centroidsNonWater = centroidsOnly[~centroidsOnly['OBJECTID'].isin(waterParcelIDS)]

In [ ]:
#https://gis.stackexchange.com/questions/222315/finding-nearest-point-in-other-geodataframe-using-geopandas
# Returns nearest neighbor in B from each point in A
def ckdnearest(gdA, gdB):
    # Converts geodataframe into np arrays
    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))

    # Creates tree
    btree = cKDTree(nB)
    # Finding nearest neighbor
    dist, idx = btree.query(nA, k=1) 

    # 
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name='dist')
        ], 
        axis=1)

    return gdf

nearestWaterIDs = ckdnearest(centroidsNonWater, centroidsOnly_water)

In [ ]:
nearestWaterIDs.set_axis(['OBJECTID',
                          'geometry',
                          'nearestID',
                          'dist'], 
                         axis=1, inplace=True)
nearestWaterIDs

,OBJECTID,geometry,nearestID,dist
0,1,POINT (-8949496.651 3176698.682),1013765,615.418975
1,2,POINT (-8949106.299 3177249.232),597436,327.743115
2,3,POINT (-8953091.749 3177318.692),646647,370.776985
3,4,POINT (-8947966.907 3177347.275),1258032,117.512084
4,5,POINT (-8949616.034 3177551.032),1019526,481.386216
...,...,...,...,...
750612,1322189,POINT (-9045381.230 3313681.126),772762,306.896621
750613,1322190,POINT (-9025984.382 3313083.411),848482,309.717701
750614,1322191,POINT (-9044224.391 3314337.698),1017530,200.662218
750615,1322192,POINT (-9004702.932 3313214.159),658412,187.822036


In [ ]:
# Generates with NaN as nearestID and dist for water parcels
gdfWithDists = gdf.join(nearestWaterIDs.drop(columns='geometry').set_index('OBJECTID'), on='OBJECTID')
gdfWithDists.fillna(value=0, inplace=True)
gdfWithDists

,OBJECTID,LEVEL3_VALUE,LDI,LSI,WMD_DISTRICT,LANDUSE_YEAR,DESCRIPTION,SHAPEAREA,SHAPELEN,geometry,centroids,nearestID,dist
0,1,1100,6.8,3.6,SJR,2013-2016,"1100: Low Density, <2 dwelling units/acre",202486.422768,2768.478185,"POLYGON ((-80.39430 27.58675, -80.39430 27.586...",POINT (-8949496.651 3176698.682),1013765.0,615.418975
1,2,1100,6.8,3.6,SJR,2013-2016,"1100: Low Density, <2 dwelling units/acre",4144.953037,287.746301,"POLYGON ((-80.39067 27.58704, -80.39066 27.587...",POINT (-8949106.299 3177249.232),597436.0,327.743115
2,3,1100,6.8,3.6,SJR,2013-2016,"1100: Low Density, <2 dwelling units/acre",9408.253317,471.548879,"POLYGON ((-80.42704 27.58723, -80.42662 27.587...",POINT (-8953091.749 3177318.692),646647.0,370.776985
3,4,1100,6.8,3.6,SJR,2013-2016,"1100: Low Density, <2 dwelling units/acre",14729.055061,639.330146,"POLYGON ((-80.37991 27.58839, -80.37987 27.588...",POINT (-8947966.907 3177347.275),1258032.0,117.512084
4,5,1100,6.8,3.6,SJR,2013-2016,"1100: Low Density, <2 dwelling units/acre",81255.766776,1202.345663,"POLYGON ((-80.39454 27.58873, -80.39374 27.588...",POINT (-8949616.034 3177551.032),1019526.0,481.386216
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338031,1338032,6460,1.0,10.0,SJR,2013-2016,6460: Mixed Scrub-shrub Wetland,8436.737078,471.816539,"POLYGON ((-82.02071 30.72787, -82.02065 30.727...",POINT (-9130491.112 3575723.293),0.0,0.000000
1338032,1338033,6460,1.0,10.0,SJR,2013-2016,6460: Mixed Scrub-shrub Wetland,4395.450443,246.783645,"POLYGON ((-81.70438 30.72415, -81.70432 30.724...",POINT (-9095303.402 3575202.887),0.0,0.000000
1338033,1338034,6460,1.0,10.0,SJR,2013-2016,6460: Mixed Scrub-shrub Wetland,303663.108819,6475.501932,"POLYGON ((-81.81171 30.72584, -81.81178 30.726...",POINT (-9106407.185 3574920.918),0.0,0.000000
1338034,1338035,6460,1.0,10.0,SJR,2013-2016,6460: Mixed Scrub-shrub Wetland,3600.924167,238.813225,"POLYGON ((-81.73858 30.72505, -81.73852 30.725...",POINT (-9099131.919 3575314.852),0.0,0.000000


In [ ]:
gdfWithDists.drop(columns='centroids').to_file("landuseLDI_centroids_nearestID_dist.geojson", driver="GeoJSON")

In [ ]:
%ls

centroidPoints.geojson  landuseLDI_centroids_nearestID_dist.geojson
data/                   README.md
demo/                   tests/
FloridaFlow/            water_centroidPoints.geojson
